In [ ]:
import os 
import tensorflow as tf 
import imageio
import matplotlib.pyplot as plt 
import numpy as np
import glob

In [ ]:
def string2img(objs, h, w):
    image_1d = []
    for obj in objs:
        image_1d.append(np.fromstring(obj, dtype=np.uint8))
    image = np.array(image_1d)
    image = image.reshape((h, w, 3))
    return image


# 設定以 gzip 壓縮
compression = tf.python_io.TFRecordCompressionType.GZIP
record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename,
                                                 options=tf.python_io.TFRecordOptions(compression))

for string_record in record_iterator:
    # 建立 Example
    example = tf.train.Example()

    # 解析來自於 TFRecords 檔案的資料
    example.ParseFromString(string_record)

    gifs, exts, csvs = None, None, None
    for i in range(4):
        # 取出 image_string 這個 Feature
        image_string1 = (example.features.feature['gif_{}'.format(i)].bytes_list.value[:])
        image_string2 = (example.features.feature['ext_{}'.format(i)].bytes_list.value[:])

        # 取出 label 這個 Feature
        label = (example.features.feature['fdb_cmd_{}'.format(i)].float_list.value[:])
        
        gif = string2img(image_string1, 240, 240)[np.newaxis, :]
        ext = string2img(image_string2, 120, 120)[np.newaxis, :]
        csv = np.array(label, dtype=np.float32)[np.newaxis, :]
        
        if gifs is None:
            gifs = gif.copy()
        else:
            gifs = np.append(gifs, gif, axis=0)
            
        if exts is None:
            exts = ext.copy()
        else:
            exts = np.append(exts, ext, axis=0)
            
        if csvs is None:
            csvs = csv.copy()
        else:
            csvs = np.append(csvs, csv, axis=0)

#     for i in range(4):
#         plt.figure(i)
#         plt.imshow(gifs[i])
#         plt.figure(i+1)
#         plt.imshow(exts[i])
#         plt.show(block=False)
#         plt.pause(0.001)
        
#     print(gifs.shape, exts.shape, csvs.shape)


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

# 圖片標準尺寸
gif_img_size = 240
ext_img_size = 120


def decode_features(features, slice_num=4):
    # convert to my structure
    giflist, extlist, csvlist = [], [], []
    for i in range(slice_num):
        gif = tf.decode_raw(features['external_{}'.format(i)], tf.float32)
        gif = tf.reshape(gif, [gif_img_size, gif_img_size, 3])
        ext = tf.decode_raw(features['eye_hand_{}'.format(i)], tf.float32)
        ext = tf.reshape(ext, [ext_img_size, ext_img_size, 3])
        csv = tf.sparse_tensor_to_dense(features['fdb_cmd_{}'.format(i)])
        csv = tf.reshape(csv, (21,))
        giflist.append(gif)
        extlist.append(ext)
        csvlist.append(csv)
    
    gifs = tf.stack(giflist, axis=0)
    exts = tf.stack(extlist, axis=0)
    csvs = tf.stack(csvlist, axis=0)
    return gifs, exts, csvs

    
def read_and_decode(filename_queue, batch_szie=32, slice_num=4):
    # 設定以 gzip 壓縮
    compression = tf.python_io.TFRecordCompressionType.GZIP
    # 建立 TFRecordReader
    reader = tf.TFRecordReader(options=tf.python_io.TFRecordOptions(
                    compression))

    # read data of tfrecords fom filename queue
    _, serialized_example = reader.read(filename_queue)

    # make features dict
    features={}
    for i in range(slice_num):
        features['external_{}'.format(i)] = tf.FixedLenFeature([], tf.string)
        features['eye_hand_{}'.format(i)] = tf.FixedLenFeature([], tf.string)
        features['fdb_cmd_{}'.format(i)] = tf.VarLenFeature(tf.float32)
        
    # read a example
    features = tf.parse_single_example(serialized_example,
        features=features)
    gifs, exts, csvs = decode_features(features, slice_num)
 
    # preprocessing
    gifs = gifs / 255.
    exts = exts / 255.
    fdbs = csvs[..., :11]
    cmds = csvs[..., 11:]

    # 打散資料順序
    batch_data = tf.train.shuffle_batch(
        [gifs, exts, fdbs, cmds],
        batch_size=batch_szie,
        capacity=64,
        num_threads=2,
        min_after_dequeue=16)

    return batch_data

In [ ]:
# 建立檔名佇列
train_queue = tf.train.string_input_producer(
  ['data.tfrecords'], num_epochs=2)

# 建立檔名佇列
valid_queue = tf.train.string_input_producer(
  ['data1.tfrecords'], num_epochs=2)

# 讀取並解析 TFRecords 的資料
train_batch = read_and_decode(train_queue)
valid_batch = read_and_decode(valid_queue, batch_szie=1)

# 初始化變數
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session()  as sess:
    sess.run(init_op)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    tc = 0
    # 示範用的簡單迴圈
    while True:
        try:
            gif_batch, ext_batch, fdb_batch, cmd_batch = sess.run(train_batch)
            print(gif_batch.shape, ext_batch.shape, fdb_batch.shape, cmd_batch.shape)
            tc += 32
        except tf.errors.OutOfRangeError:
            print('t finish', tc)
            break
    vc = 0
    while True:
        try:
            gif_batch, ext_batch, fdb_batch, cmd_batch = sess.run(valid_batch)
            print(gif_batch.shape, ext_batch.shape, fdb_batch.shape, cmd_batch.shape)
            vc += 1
        except tf.errors.OutOfRangeError:
            print('v finish', vc)
            break    
            
    coord.request_stop()
    coord.join(threads)